# Titanic: Machine Learning from Disaster

Predict survival on the Titanic

## Description

The sinking of the RMS Titanic is ine f the most infamous shipwrecks in history.On **April 15, 1912**, during her maiden voyage, the Titanic sank after colliding with an iceberg,killing **1,502** out of **2,224** passengers and crew.This sensational tragedy shocked the international community and led to better safety regulations for ships. 

- One of the reasons that the shipwreck led to such loss of life was that:
    - there were not enough lifeboats for the passengers and crew.
    - Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as:
        - women, children, and the upper-class.

## Goal:

Apply machine learning techniques to successfully predict which passengers survived the sinking of the Titanic

## Machine Learning "Binary Classification"

- Use pandas for data manipulation
- Use matplotlib and seaborn for data visualization
- Learn to build models with scikit-learn
- Use cross-validation to make sure your model generalizes to new data (i.e., it doesn’t “overfit”)
- Use parameter tuning and grid search to select the best performing model out of several different classification algorithms

### Feature Engineering

Using existing features to create new features to determine if they provide new signals to predict our outcome. For this dataset, we will create a title feature to determine if it played a role in survival.

### Metric

Score is the percentage of passengers that correctly predict. This is known simply as **"accuracy”**.

### XGBoost

- Learn how to use the extremely popular XGBoost algorithm.

### Ensembling/Stacking:

- Use the fundamental skill of “ensembling” to combine the predictions of several models

# Exploratory Data Analysis (EDA)

With a dataset of 891 passenger samples for our training set and their associated labels containing features like sex, age, and class etc.. attempt to predict the survivors of a small test group of 418 samples in the same format.

|Variable Name | Description
|:-------------:|:-------------|
| PassengerId | Passenger's id
| Survived | Survived (1) or died (0)
| Pclass | Passenger’s class (1 = Upper, 2 = Middle, 3 = Lower)
| Name | Passenger’s name
| Sex| Passenger’s sex
| Age| Passenger’s age
| SibSp | Number of siblings/spouses aboard
| Parch | Number of parents/children aboard
| Ticket | Ticket number
| Fare | Passenger’s fare
| Cabin | Cabin number
| Embarked | Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

In [1]:
# Data analysis and wrangling
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc

# Configure visualisations
%matplotlib inline
plt.style.use('ggplot')

# Configure seaborn
sns.set_style('white',{"xtick.major.size": 5, "ytick.major.size": 5})


plt.rcParams['figure.figsize'] = (15,6)
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 18

import warnings
warnings.filterwarnings('ignore')

In [2]:
# The training set should be used to build your machine learning models.
train_df = pd.read_csv('train.csv')

# The test set should be used to see how well your model performs on unseen data. 
test_df = pd.read_csv('test.csv')

In [3]:
print(train_df.shape, test_df.shape)

(891, 12) (418, 11)


In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train_df['Sex'] = train_df['Sex'].str.capitalize()
test_df['Sex'] = test_df['Sex'].str.capitalize()

In [6]:
# Combine train and test data
all_data = pd.concat([train_df, test_df], ignore_index=True)

In [7]:
all_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",Male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",Female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",Male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",Male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",Female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",Male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,NaN,3,"Ware, Mr. Frederick",Male,NaN,0,0,359309,8.0500,NaN,S


In [8]:
# Features are available in dataset
print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


### Data Types

#### Categorical features
- **Nominal:** Survived, Sex and Embarked
- **Ordinal:** Pclass

#### Numerical Features
- **Continous:** Age, Fare
- **Discrete:** SibSp, Parch

#### Features are mixed data types

Numerical, alphanumeric data within same feature.

- **Ticket**: is a mix of numeric and alphanumeric data types.
- **Cabin**: is alphanumeric.

Total samples are **891** or **40%** of the actual number of passengers on board the Titanic.

In [12]:
train_df.info()
print("--"*25)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0  

### Missing Values
- **Age,Cabin,Embarked and Fare** features contains blank,null or empty values.

In [14]:
all_data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",Male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",Female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [48]:
missingValue = pd.DataFrame(all_data.iloc[:,2:].isnull().sum(axis=0).sort_values(ascending=False),columns=['NaN'])
missingValue["Percentage"]= missingValue.NaN.apply(lambda x: '{:.2f}%'.format((float(x)/all_data.shape[0])*100))
missingValue[missingValue.NaN > 0] 

,NaN,Percentage
Cabin,1014,77.46%
Age,263,20.09%
Embarked,2,0.15%
Fare,1,0.08%


### Analyzing Features

Observations for all features:

- **Sex**: The chance of survival for women is high as compared to men.

- **Pclass**: Higher socio-economic status passenger survived more than others. Being a first class passenger have better chances of survival. The survival rate for Pclass3 is very low. For women, the chance of survival from Pclass1 is almost 1 and is high too for those from Pclass2.

- **Age**: Younger passenger survived more than other passengers like children less than 5-10 years do have a high chance of survival. Passengers between age group 15 to 35 died a lot.

- **Embarked**: This is a very interesting feature. The chances of survival at C looks to be better than even though the majority of Pclass1 passengers got up at S. Passengers at Q were all from Pclass3.

- **Parch + SibSp** : Having 1-2 siblings,spouse on board or 1-3 Parents shows a greater chance of probablity rather than being alone or having a large family travelling with you.

### Taget Variable

- **Survived**

#### How many survived?

It is evident that not many passengers survived the accident

- Out of **891** passengers in training set, only **342** survived **38.4%** of the total training set survived.